<a href="https://colab.research.google.com/github/jaysmerrill/Yaquina_Bay_Seiching/blob/main/seiche_prediction_rtdata_optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# Real-time forcings (no parquet) + var_wl (20–30 min bandpass → X-hour variance)
# Sources: NOAA CO-OPS 9435380 (WL, wind, air pressure), NDBC 46050 (stdmet, swden), HMSC (baro gap-fill)
# ============================================================
import io, gzip, requests, re, json, os, math, time
import numpy as np
import pandas as pd

# plotting + signal
!pip -q install plotly scipy
from scipy.signal import butter, filtfilt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# -------------------
# USER KNOBS
# -------------------
YEARS                = list(range(2008, pd.Timestamp.utcnow().year + 1))
DT_MINUTES           = 6  # common model grid
WL_BAND_MIN_MIN      = 20.0
WL_BAND_MAX_MIN      = 30.0
VAR_WIN_HOURS        = 2.0     # <-- default window for variance of band-passed WL (tweakable)
NOAA_STATION         = "9435380"  # Newport / South Beach
NOAA_BEGIN_DATE      = "20080101"
NOAA_END_DATE        = pd.Timestamp.utcnow().strftime("%Y%m%d")
NOAA_CHUNK_DAYS      = 365

# HMSC monthly archive (baro gap fill for NOAA air_pressure ≥3h gaps; <3h: spline)
HMSC_UTC_OFFSET_H    = +8
INHG_TO_PA           = 3386.389

# NDBC buoy 46050
BUOY                 = "46050"

# Optional preview window for plots
PRED_START           = None  # e.g., pd.Timestamp("2022-11-01")
PRED_END             = None  # e.g., pd.Timestamp("2022-12-15")

# -------------------
# HTTP session with retries
# -------------------
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def _make_session():
    s = requests.Session()
    retry = Retry(total=4, backoff_factor=0.6, status_forcelist=(429,500,502,503,504),
                  allowed_methods=["HEAD","GET","OPTIONS"], raise_on_status=False)
    s.headers.update({"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) RealtimeForcings/1.0", "Accept":"*/*"})
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://", HTTPAdapter(max_retries=retry))
    return s
session = _make_session()

def _datestr(dt) -> str:
    return pd.Timestamp(dt).strftime("%Y%m%d")

# -------------------
# NOAA CO-OPS: Water Level (6-min) + bandpass(20–30min) + centered rolling variance -> var_wl
# -------------------
def parse_noaa_wl_csv(txt:str)->pd.DataFrame:
    if txt.strip().lower().startswith("error"):
        return pd.DataFrame(columns=["time","wl"])
    df = pd.read_csv(io.StringIO(txt))
    if df.shape[1] < 2: return pd.DataFrame(columns=["time","wl"])
    df.columns = [str(c).strip().lower().replace("  "," ") for c in df.columns]
    tcol=None
    for k in ("date time","date_time","time","t","date"):
        if k in df.columns: tcol=k; break
    if tcol is None: return pd.DataFrame(columns=["time","wl"])
    df["time"] = pd.to_datetime(df[tcol], errors="coerce", utc=True).dt.tz_localize(None)
    vcol=None
    for k in ("water level","water_level","wl","v"):
        if k in df.columns: vcol=k; break
    if vcol is None: vcol = df.columns[1]
    df["wl"] = pd.to_numeric(df[vcol], errors="coerce")
    out = df[["time","wl"]].dropna(subset=["time"]).sort_values("time").drop_duplicates("time")
    out.loc[(out["wl"]<-50)|(out["wl"]>50),"wl"]=np.nan
    return out.reset_index(drop=True)

def fetch_noaa_wl(begin_date:str, end_date:str, station:str, days_per_chunk:int=31)->pd.DataFrame:
    start = pd.to_datetime(begin_date, format="%Y%m%d"); end = pd.to_datetime(end_date, format="%Y%m%d")
    frames=[]
    cur=start
    while cur <= end:
        chunk_end = min(end, cur + pd.Timedelta(days=days_per_chunk-1))
        url = ("https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
               f"?begin_date={_datestr(cur)}&end_date={_datestr(chunk_end)}"
               f"&station={station}&product=water_level&datum=MLLW&time_zone=gmt&units=metric"
               f"&interval=6&application=RealtimeForcings&format=csv")
        try:
            r = session.get(url, timeout=90)
            if r.status_code==200 and r.content:
                dfc = parse_noaa_wl_csv(r.content.decode("utf-8","ignore"))
                if not dfc.empty: frames.append(dfc)
        except: pass
        cur = chunk_end + pd.Timedelta(days=1)
    if not frames: return pd.DataFrame(columns=["time","wl"])
    return pd.concat(frames, ignore_index=True).sort_values("time").drop_duplicates("time").reset_index(drop=True)

def bandpass_then_var(series:pd.DataFrame, val_col:str, dt_minutes:int,
                      band_min_min:float, band_max_min:float, var_win_h:float)->pd.DataFrame:
    if series.empty: return pd.DataFrame(columns=["time","var_wl"])
    t0,t1 = series["time"].min(), series["time"].max()
    grid = pd.date_range(t0.floor(f"{dt_minutes}min"), t1.ceil(f"{dt_minutes}min"), freq=f"{dt_minutes}min")
    s = series.set_index("time")[val_col].reindex(grid)
    s = s.interpolate(method="time", limit_area="inside")
    dt_sec = dt_minutes*60.0; fs=1.0/dt_sec
    f_lo = 1.0/(band_max_min*60.0)  # low cutoff = longer period edge
    f_hi = 1.0/(band_min_min*60.0)  # high cutoff = shorter period edge
    wn = [max(1e-6, f_lo/(fs/2.0)), min(0.999, f_hi/(fs/2.0))]
    b,a = butter(6, wn, btype="bandpass")
    x = s.to_numpy(dtype=float)
    if np.isnan(x).any():
        idx = np.arange(len(x)); good = np.isfinite(x);
        if good.any(): x[~good] = np.interp(idx[~good], idx[good], x[good])
    x_bp = filtfilt(b,a,x,method="gust")  # zero-phase
    win = max(3, int(round((var_win_h*60.0)/dt_minutes)))
    minp = max(2, int(0.8*win))
    var_series = pd.Series(x_bp, index=grid).rolling(window=win, center=True, min_periods=minp).var()
    return pd.DataFrame({"time":grid, "var_wl":var_series.astype("float32")}).dropna(subset=["time"]).reset_index(drop=True)

df_wl6  = fetch_noaa_wl(NOAA_BEGIN_DATE, NOAA_END_DATE, NOAA_STATION, days_per_chunk=31)
df_varWL = bandpass_then_var(df_wl6, "wl", DT_MINUTES, WL_BAND_MIN_MIN, WL_BAND_MAX_MIN, VAR_WIN_HOURS)
print("WL coverage:", (df_wl6["time"].min() if len(df_wl6) else None), "→", (df_wl6["time"].max() if len(df_wl6) else None), f"(n={len(df_wl6):,})")

# -------------------
# NOAA CO-OPS: Wind (hourly)
# -------------------
def parse_noaa_wind_csv(txt:str)->pd.DataFrame:
    df = pd.read_csv(io.StringIO(txt))
    tcol = [c for c in df.columns if str(c).strip().lower() in ("t","time","date time","date_time","date")]
    tcol = tcol[0] if tcol else df.columns[0]
    df["time"] = pd.to_datetime(df[tcol], errors="coerce", utc=True).dt.tz_localize(None)
    def _find(keys):
        keys = [k.lower() for k in keys]
        for c in df.columns:
            s = str(c).lower()
            if any(k == s or k in s for k in keys): return c
        return None
    sp_col = _find(["s","speed"])
    dir_col= _find(["d","dir","direction"])
    out = pd.DataFrame({"time":df["time"]})
    out["wind9435380_speed"] = pd.to_numeric(df[sp_col], errors="coerce") if sp_col else np.nan
    out["wind9435380_dir"]   = pd.to_numeric(df[dir_col], errors="coerce") if dir_col else np.nan
    out.loc[(out["wind9435380_dir"]<0)|(out["wind9435380_dir"]>360),"wind9435380_dir"]=np.nan
    out.loc[(out["wind9435380_speed"]<0)|(out["wind9435380_speed"]>100),"wind9435380_speed"]=np.nan
    return (out.sort_values("time").dropna(subset=["time"])
            .drop_duplicates("time").reset_index(drop=True))

def fetch_noaa_wind_dataframe(begin_date:str, end_date:str, station:str, days_per_chunk:int=365)->pd.DataFrame:
    start = pd.to_datetime(begin_date, format="%Y%m%d"); end = pd.to_datetime(end_date, format="%Y%m%d")
    frames=[]; cur=start
    while cur <= end:
        chunk_end = min(end, cur + pd.Timedelta(days=days_per_chunk-1))
        url = ("https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
               f"?begin_date={_datestr(cur)}&end_date={_datestr(chunk_end)}"
               f"&station={station}&product=wind&time_zone=gmt&interval=h&units=metric"
               "&application=RealtimeForcings&format=csv")
        try:
            r = session.get(url, timeout=60)
            if r.status_code==200 and r.content:
                dfc = parse_noaa_wind_csv(r.content.decode("utf-8","ignore"))
                if not dfc.empty: frames.append(dfc)
        except: pass
        cur = chunk_end + pd.Timedelta(days=1)
    if not frames: return pd.DataFrame(columns=["time","wind9435380_speed","wind9435380_dir"])
    return pd.concat(frames, ignore_index=True).sort_values("time").drop_duplicates("time").reset_index(drop=True)

df_wind = fetch_noaa_wind_dataframe(NOAA_BEGIN_DATE, NOAA_END_DATE, NOAA_STATION, days_per_chunk=NOAA_CHUNK_DAYS)
print("Wind coverage:", (df_wind["time"].min() if len(df_wind) else None), "→", (df_wind["time"].max() if len(df_wind) else None), f"(n={len(df_wind):,})")

# -------------------
# NOAA CO-OPS: Air Pressure (hourly) + HMSC gap-fill logic
# -------------------
def parse_noaa_air_pressure_csv(txt: str) -> pd.DataFrame:
    df = pd.read_csv(io.StringIO(txt))
    # time col
    tcol = None
    for k in ("t","time","date time","date_time","date"):
        m = [c for c in df.columns if str(c).strip().lower()==k]
        if m: tcol = m[0]; break
    if tcol is None: tcol = df.columns[0]
    # value col
    vcol = None
    for key in ("air_pressure","air pressure","barometric pressure","baro","pressure"):
        m = [c for c in df.columns if key in str(c).strip().lower()]
        if m: vcol = m[0]; break
    if vcol is None: vcol = df.columns[1] if df.shape[1] > 1 else df.columns[0]
    out = pd.DataFrame({
        "time": pd.to_datetime(df[tcol], errors="coerce", utc=True).dt.tz_localize(None),
        "air_pressure_hpa": pd.to_numeric(df[vcol], errors="coerce")
    }).dropna(subset=["time"]).sort_values("time").drop_duplicates("time").reset_index(drop=True)
    out.loc[(out["air_pressure_hpa"] < 850) | (out["air_pressure_hpa"] > 1100), "air_pressure_hpa"] = np.nan
    return out

def fetch_noaa_air_pressure_dataframe(begin_date: str, end_date: str, station: str,
                                      days_per_chunk: int = 365) -> pd.DataFrame:
    start = pd.to_datetime(begin_date, format="%Y%m%d"); end = pd.to_datetime(end_date, format="%Y%m%d")
    frames=[]; cur = start
    while cur <= end:
        chunk_end = min(end, cur + pd.Timedelta(days=days_per_chunk-1))
        url = ("https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
               f"?begin_date={_datestr(cur)}&end_date={_datestr(chunk_end)}"
               f"&station={station}&product=air_pressure&time_zone=gmt&interval=h&units=metric"
               "&application=RealtimeForcings&format=csv")
        try:
            r = session.get(url, timeout=60)
            if r.status_code==200 and r.content:
                dfc = parse_noaa_air_pressure_csv(r.content.decode("utf-8","ignore"))
                if not dfc.empty: frames.append(dfc)
        except: pass
        cur = chunk_end + pd.Timedelta(days=1)
    if not frames: return pd.DataFrame(columns=["time","air_pressure_hpa"])
    return (pd.concat(frames, ignore_index=True)
            .sort_values("time").drop_duplicates("time").reset_index(drop=True))

# HMSC baro (AvgBP in inHg) -> Pa for gap fill
def fetch_hmsc_month_text(year:int, month:int) -> str|None:
    url = f"http://weather.hmsc.oregonstate.edu/weather/weatherproject/archive/{year}/HMSC_{year}{month:02d}.dat"
    try:
        r = session.get(url, timeout=60)
        if r.status_code==200 and r.content:
            return r.content.decode("utf-8", "ignore")
    except: pass
    return None

def parse_hmsc_dat_avgbp_pa(txt: str) -> pd.DataFrame:
    """Parse HMSC monthly .dat text to dataframe with UTC 'time' and pressure in Pa.

    - Accepts files where the first CSV header line starts with TIMESTAMP
    - Filters out the two metadata/unit rows that appear after the header
    - Parses local (America/Los_Angeles) times and converts to UTC (naive)
    - Converts AvgBP (inHg) to Pascals
    """
    # 1) Find the CSV header (TIMESTAMP,...)
    lines = txt.splitlines()
    start_idx = None
    for i, ln in enumerate(lines):
        if ln.strip().strip('"').startswith("TIMESTAMP"):
            start_idx = i
            break
    if start_idx is None:
        return pd.DataFrame(columns=["time", "bp_pa"])

    content = "\n".join(lines[start_idx:])
    df = pd.read_csv(io.StringIO(content))

    # 2) First column is time text; strip quotes and whitespace
    raw_t = df.iloc[:, 0].astype(str).str.strip().str.replace('"', '', regex=False)

    # Filter out the two metadata/unit rows (keep only rows that look like YYYY-mm-dd HH:MM:SS[.fff])
    time_like = raw_t.str.match(r"\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}(\.\d+)?$")
    raw_t = raw_t.where(time_like)  # non-matching rows -> NaN

    # 3) Fast explicit parse, then gentle fallbacks
    t = pd.to_datetime(raw_t, format="%Y-%m-%d %H:%M:%S", errors="coerce")
    if t.isna().any():
        t_ms = pd.to_datetime(raw_t, format="%Y-%m-%d %H:%M:%S.%f", errors="coerce")
        t = t.fillna(t_ms)
    if t.isna().any():
        # Last resort (rare)
        t = t.fillna(pd.to_datetime(raw_t, errors="coerce"))

    # Local (America/Los_Angeles) -> UTC (naive)
    try:
        t = (
            t.dt.tz_localize("America/Los_Angeles", nonexistent="shift_forward", ambiguous="NaT")
             .dt.tz_convert("UTC")
             .dt.tz_localize(None)
        )
    except Exception:
        # If tz database isn't available in the environment, proceed as naive
        pass

    # 4) Find AvgBP (inHg), convert to float
    target = None
    for c in df.columns:
        if "avgbp" in str(c).lower():
            target = c
            break
    if target is None:
        return pd.DataFrame(columns=["time", "bp_pa"])

    avgbp_inhg = pd.to_numeric(df[target], errors="coerce")

    # Avoid the invalid value warnings: use between() with inclusive bounds
    good = avgbp_inhg.between(26.0, 33, inclusive="both")
    avgbp_inhg = avgbp_inhg.where(good)

    INHG_TO_PA = 3386.389  # Pa per inHg
    bp_pa = avgbp_inhg * INHG_TO_PA

    out = (
        pd.DataFrame({"time": t, "bp_pa": bp_pa})
        .dropna(subset=["time"])
        .astype({"bp_pa": "float64"})
        .sort_values("time")           # now safe: all datetimes
        .drop_duplicates("time")
        .reset_index(drop=True)
    )
    return out


def month_iter(start:pd.Timestamp, end:pd.Timestamp):
    y,m = start.year, start.month
    while pd.Timestamp(year=y, month=m, day=1) <= end:
        yield (y,m)
        if m==12: y,m=y+1,1
        else: m+=1

# Fetch + merge air pressure with HMSC fill
df_air = fetch_noaa_air_pressure_dataframe(NOAA_BEGIN_DATE, NOAA_END_DATE, NOAA_STATION, days_per_chunk=NOAA_CHUNK_DAYS)
if len(df_air):
    df_air["air_pressure_pa"] = (df_air["air_pressure_hpa"].astype(float) * 100.0).astype("float64")
    ap_start, ap_end = df_air["time"].min(), df_air["time"].max()
    hourly_grid = pd.date_range(ap_start.floor("h"), ap_end.ceil("h"), freq="h")
    s_noaa = df_air.set_index("time")["air_pressure_pa"].reindex(hourly_grid).astype("float64")

    # HMSC coverage padded
    start_pad = (ap_start - pd.Timedelta(days=31)).normalize()
    end_pad   = (ap_end   + pd.Timedelta(days=31)).normalize()
    hmsc_frames=[]
    for y,m in month_iter(start_pad, end_pad):
        txt = fetch_hmsc_month_text(y,m)
        if not txt: continue
        dfm = parse_hmsc_dat_avgbp_pa(txt)
        if not dfm.empty:
            mask = (dfm["time"]>=start_pad) & (dfm["time"]<=end_pad)
            hmsc_frames.append(dfm.loc[mask])
    df_hmsc_bp = (pd.concat(hmsc_frames, ignore_index=True)
                  .sort_values("time").drop_duplicates("time").reset_index(drop=True)) if hmsc_frames else pd.DataFrame(columns=["time","bp_pa"])
    if len(df_hmsc_bp):
        s_hmsc = (df_hmsc_bp.set_index("time")["bp_pa"].reindex(hourly_grid)
                  .interpolate(method="time", limit_area="inside").astype("float64"))
    else:
        s_hmsc = pd.Series(index=hourly_grid, dtype="float64")

    # fill rules
    filled = s_noaa.copy()
    nan_mask = filled.isna().to_numpy()
    if nan_mask.any():
        starts = np.where(~nan_mask[:-1] & nan_mask[1:])[0] + 1 if len(nan_mask)>1 else np.array([],dtype=int)
        ends   = np.where(nan_mask[:-1] & ~nan_mask[1:])[0] + 1 if len(nan_mask)>1 else np.array([],dtype=int)
        if nan_mask[0]:  starts = np.r_[0, starts]
        if nan_mask[-1]: ends   = np.r_[ends, len(nan_mask)]
        for s,e in zip(starts, ends):
            run_len = e - s
            if run_len >= 3:  # >=3h: fill from HMSC
                seg_idx = hourly_grid[s:e]
                filled.iloc[s:e] = s_hmsc.loc[seg_idx].values
        if filled.isna().any():
            try:
                filled = filled.interpolate(method="spline", order=3, limit_area="inside")
            except Exception:
                filled = filled.interpolate(method="time", limit_area="inside")
    df_air_merged = pd.DataFrame({"time": hourly_grid, "air_pressure_pa": filled.to_numpy()})
else:
    df_air_merged = pd.DataFrame(columns=["time","air_pressure_pa"])

print("Air pressure coverage:", (df_air_merged["time"].min() if len(df_air_merged) else None),
      "→", (df_air_merged["time"].max() if len(df_air_merged) else None),
      f"(n={len(df_air_merged):,})")

# -------------------
# NDBC 46050: stdmet (Hs, DPD, APD, MWD) + SWDEN bands (high_HIG, Hswell, Hsea)
# -------------------
def _ndbc_to_float(tok:str) -> float:
    s = str(tok).strip()
    if s == "" or s.upper() == "MM": return np.nan
    if re.fullmatch(r"-?9+(\.0+)?", s): return np.nan
    try: return float(s)
    except: return np.nan

STDMET_PHP_PAT = f"https://www.ndbc.noaa.gov/view_text_file.php?filename={BUOY}h"+"{year}.txt.gz&dir=data/historical/stdmet/"

def fetch_stdmet_year(year:int)->str|None:
    for url in (STDMET_PHP_PAT.format(year=year),):
        try:
            r = session.get(url, timeout=40)
            if r.status_code==200 and r.text and "Page Not Found" not in r.text:
                return r.text
        except: pass
    return None

def parse_stdmet_text(txt:str)->pd.DataFrame:
    rows=[]
    for ln in txt.splitlines():
        if not ln or ln.lstrip().startswith("#"): continue
        parts = re.split(r"\s+", ln.strip())
        if len(parts) < 6: continue
        y,mo,dy,hh,mi = parts[:5]
        try:
            y=int(y); mo=int(mo); dy=int(dy); hh=int(hh); mi=int(mi)
            year = 2000+y if y<100 else y
            ts = pd.Timestamp(year=year, month=mo, day=dy, hour=hh, minute=mi, tz="UTC").tz_localize(None)
        except: continue
        vals = parts[5:]
        if len(vals) < 13: vals = (vals + ["MM"]*13)[:13]
        vals = [_ndbc_to_float(v) for v in vals]
        rows.append([ts] + vals)
    if not rows: return pd.DataFrame()
    cols = ["time","WDIR","WSPD","GST","WVHT","DPD","APD","MWD","PRES","ATMP","WTMP","DEWP","VIS","TIDE"]
    df = pd.DataFrame(rows, columns=cols).sort_values("time").drop_duplicates("time")
    df = df.rename(columns={"WVHT":"Hs"}).reset_index(drop=True)
    for c,thr in [("Hs",50.0),("DPD",50.0),("APD",50.0)]:
        if c in df.columns: df.loc[df[c]>thr, c]=np.nan
    df.loc[(df["MWD"]<0)|(df["MWD"]>360),"MWD"]=np.nan
    keep = ["time","Hs","DPD","APD","MWD"]
    for k in keep:
        if k not in df.columns: df[k]=np.nan
    return df[keep]

def build_stdmet(years: list[int]) -> pd.DataFrame:
    frames=[]
    for y in years:
        txt = fetch_stdmet_year(y)
        if not txt: continue
        dfy = parse_stdmet_text(txt)
        if not dfy.empty: frames.append(dfy)
    if not frames: return pd.DataFrame(columns=["time","Hs","DPD","APD","MWD"])
    return pd.concat(frames, ignore_index=True).sort_values("time").drop_duplicates("time").reset_index(drop=True)

# SWDEN
SWDEN_PHP_PATS = [
    f"https://www.ndbc.noaa.gov/view_text_file.php?filename={BUOY}w"+"{year}.txt.gz&dir=data/historical/swden/",
    f"https://www.ndbc.noaa.gov/view_text_file.php?filename={BUOY}w"+"{year}.txt.gz&dir=data/swden/"+BUOY+"/",
]
def fetch_swden_year(year:int)->str|None:
    for url in SWDEN_PHP_PATS:
        try:
            r = session.get(url.format(year=year), timeout=40)
            if r.status_code==200 and r.text and "Page Not Found" not in r.text:
                return r.text
        except: pass
    return None

def split_swden_blocks(lines:list[str]):
    header=[]; i=0
    while i<len(lines) and lines[i].startswith("#"): header.append(lines[i]); i+=1
    while i<len(lines) and not lines[i].strip(): i+=1
    freq_line = lines[i].rstrip("\n") if i<len(lines) else None
    i += 1 if i<len(lines) else 0
    data=[ln for ln in lines[i:] if ln.strip()]
    return header, freq_line, data

def parse_swden_year(txt:str)->pd.DataFrame:
    lines = txt.splitlines()
    _, freq_line, data_lines = split_swden_blocks(lines)
    if not freq_line: return pd.DataFrame()
    freq_tokens = [t for t in re.split(r"\s+", freq_line.strip()) if t]
    if any(k in freq_tokens[:6] for k in ("YY","MM","DD","hh")):
        idx = lines.index(freq_line)
        j=idx+1
        while j<len(lines) and not lines[j].strip(): j+=1
        if j<len(lines):
            freq_tokens = [t for t in re.split(r"\s+", lines[j].strip()) if t]
            data_lines  = [ln for ln in lines[j+1:] if ln.strip()]
        else:
            return pd.DataFrame()
    def _num(tok):
        try: return float(tok)
        except: return np.nan
    freqs = np.array([_num(t) for t in freq_tokens], dtype=float)
    freqs = freqs[np.isfinite(freqs)]
    if freqs.size==0: return pd.DataFrame()
    dfw = np.empty_like(freqs)
    if freqs.size==1:
        dfw[:] = 0.0
    else:
        dfw[1:-1] = 0.5*(freqs[2:] - freqs[:-2])
        dfw[0]    = freqs[1] - freqs[0]
        dfw[-1]   = freqs[-1] - freqs[-2]
        dfw = np.where(dfw<0, np.nan, dfw)
    mask_hig   = (freqs < 0.05)
    mask_swell = (freqs >= 0.05) & (freqs <= 0.14)
    mask_sea   = (freqs > 0.14) & (freqs <= 0.30)
    rows=[]
    for ln in data_lines:
        parts = re.split(r"\s+", ln.strip())
        if len(parts) < 4: continue
        y,mo,dy,hh = parts[:4]
        try:
            y=int(y); mo=int(mo); dy=int(dy); hh=int(hh)
            year = 2000+y if y<100 else y
            ts = pd.Timestamp(year=year, month=mo, day=dy, hour=hh, tz="UTC").tz_localize(None)
        except: continue
        vals = np.array([_ndbc_to_float(v) for v in parts[4:]], dtype=float)
        if vals.size != freqs.size:
            if vals.size < freqs.size: vals = np.pad(vals, (0, freqs.size - vals.size), constant_values=np.nan)
            else: vals = vals[:freqs.size]
        def m0(mask): return np.nansum(vals[mask] * dfw[mask]) if mask.any() else np.nan
        high_HIG = 4.0*np.sqrt(m0(mask_hig))   if mask_hig.any()   else np.nan
        Hswell   = 4.0*np.sqrt(m0(mask_swell)) if mask_swell.any() else np.nan
        Hsea     = 4.0*np.sqrt(m0(mask_sea))   if mask_sea.any()   else np.nan
        rows.append((ts, high_HIG, Hswell, Hsea))
    if not rows: return pd.DataFrame()
    return (pd.DataFrame(rows, columns=["time","high_HIG","Hswell","Hsea"])
            .sort_values("time").drop_duplicates("time").reset_index(drop=True))

df_stdmet = build_stdmet(YEARS)
df_swden  = pd.concat([parse_swden_year(txt) for y in YEARS if (txt:=fetch_swden_year(y))], ignore_index=True) \
               if YEARS else pd.DataFrame(columns=["time","high_HIG","Hswell","Hsea"])

# -------------------
# Common 6-min grid (based on available sources)
# -------------------
def _interp_timegrid(t_src:pd.Series, x_src:np.ndarray, t_grid:pd.DatetimeIndex)->np.ndarray:
    s = pd.Series(x_src, index=pd.to_datetime(t_src))
    u = s.reindex(pd.to_datetime(sorted(set(s.index).union(set(t_grid)))))
    u = u.interpolate(method="time", limit_area="inside")
    return u.reindex(pd.to_datetime(t_grid)).to_numpy()

t0_candidates=[]; t1_candidates=[]
for d,col in [(df_varWL,"time"), (df_wl6,"time"), (df_wind,"time"), (df_air_merged,"time"),
              (df_stdmet,"time"), (df_swden,"time")]:
    if len(d):
        t0_candidates.append(d[col].min()); t1_candidates.append(d[col].max())
if not t0_candidates:
    raise RuntimeError("No sources available; check API windows.")
t0 = max(t0_candidates); t1 = min(t1_candidates)
t_grid = pd.date_range(t0, t1, freq=f"{DT_MINUTES}min")

# -------------------
# Assemble seq_mat on grid (NO parquet; all online)
# -------------------
seq_mat = {}

# target: var_wl (requested)
seq_mat["var_wl"] = _interp_timegrid(df_varWL["time"], df_varWL["var_wl"].values, t_grid)

# WL raw (optional for QC)
seq_mat["wl"] = _interp_timegrid(df_wl6["time"], df_wl6["wl"].values, t_grid) if len(df_wl6) else np.full(len(t_grid), np.nan)

# wind to 6-min grid
if len(df_wind):
    seq_mat["wind9435380_speed"] = _interp_timegrid(df_wind["time"], df_wind["wind9435380_speed"].values, t_grid)
    seq_mat["wind9435380_dir"]   = _interp_timegrid(df_wind["time"], df_wind["wind9435380_dir"].values,   t_grid)

# air pressure (Pa) on 6-min grid
if len(df_air_merged):
    seq_mat["air_pressure_pa"] = _interp_timegrid(df_air_merged["time"], df_air_merged["air_pressure_pa"].values, t_grid)

# 46050 stdmet (EXCLUDE Hs from features, but keep for QC if you want)
if len(df_stdmet):
    for c in ["DPD","APD","MWD","Hs"]:
        seq_mat[c] = _interp_timegrid(df_stdmet["time"], df_stdmet[c].values, t_grid)

# 46050 swden bands
if len(df_swden):
    for c in ["high_HIG","Hswell","Hsea"]:
        seq_mat[c] = _interp_timegrid(df_swden["time"], df_swden[c].values, t_grid)

# Final dataframe
df_all = pd.DataFrame({"time": t_grid})
for k,v in seq_mat.items():
    df_all[k] = v

# -------------------
# Interactive, linked plots (QC)
# -------------------
def _col_in(df, name): return (name in df.columns) and pd.api.types.is_numeric_dtype(df[name])
def _pick(df, names): return [c for c in names if _col_in(df, c)]

def _stacked_subplot(df, cols, title, start=None, end=None):
    if not cols:
        fig = go.FigureWidget(); fig.update_layout(title=f"{title} (no series)", height=150); return fig
    d = df.sort_values("time")
    if start is not None: d = d[d["time"] >= pd.Timestamp(start)]
    if end   is not None: d = d[d["time"] <= pd.Timestamp(end)]
    rows=len(cols)
    fig = make_subplots(rows=rows, cols=1, shared_xaxes=True, vertical_spacing=0.007,
                        row_heights=[1.0/rows]*rows)
    fig = go.FigureWidget(fig)
    for i,c in enumerate(cols, start=1):
        y = d[c]
        fig.add_trace(go.Scattergl(x=d["time"], y=y, name=c, mode="lines",
                                   line=dict(width=1),
                                   hovertemplate="%{x|%Y-%m-%d %H:%M}<br>"+c+": %{y:.4g}<extra></extra>"),
                      row=i, col=1)
        fig.update_yaxes(title_text=c, row=i, col=1)
    fig.update_layout(title=title, height=max(280, rows*85), hovermode="x unified",
                      margin=dict(t=40,b=10,l=60,r=15))
    fig.update_xaxes(rangeslider=dict(visible=True), row=rows, col=1)
    return fig

def plot_forcings_linked(df_all, start=None, end=None,
                         group_layout=("Target","Waves","Wind","Pressure","Other")):
    assert "time" in df_all.columns
    candidates = [c for c in df_all.columns if c!="time" and pd.api.types.is_numeric_dtype(df_all[c])]
    # Buckets
    target   = _pick(df_all, ["var_wl","wl"])
    waves    = _pick(df_all, ["high_HIG","Hswell","Hsea","DPD","APD","MWD","Hs"])
    wind     = [c for c in candidates if c.startswith("wind9435380")]
    pressure = _pick(df_all, ["air_pressure_pa"])
    other    = [c for c in candidates if c not in set(target)|set(waves)|set(wind)|set(pressure)]
    groups = {"Target":target, "Waves":waves, "Wind":wind, "Pressure":pressure, "Other":other}

    figs=[]
    for g in group_layout:
        cols = groups.get(g, [])
        figs.append(_stacked_subplot(df_all, cols, f"{g}", start=start, end=end))

    # link x across figures
    syncing={"busy":False}
    def _link(src,*others):
        def _on_relayout(_, relayout_data):
            if syncing["busy"]: return
            if not any(k.startswith("xaxis.range") or k=="xaxis.autorange" for k in relayout_data.keys()):
                return
            x0 = relayout_data.get("xaxis.range[0]"); x1 = relayout_data.get("xaxis.range[1]")
            autor = relayout_data.get("xaxis.autorange")
            syncing["busy"]=True
            try:
                for f in others:
                    if autor is True: f.update_xaxes(autorange=True)
                    elif x0 is not None and x1 is not None: f.update_xaxes(range=[x0,x1])
            finally:
                syncing["busy"]=False
        src.on_relayout(_on_relayout)
    for i,src in enumerate(figs): _link(src, *[f for j,f in enumerate(figs) if j!=i])
    for f in figs: display(f)
    return figs

_ = plot_forcings_linked(df_all, start=PRED_START, end=PRED_END,
                         group_layout=("Target","Waves","Wind","Pressure"))

# ------------------------------
# ⛔ Hard stop (QC before any ML)
# ------------------------------
STOP_BEFORE_ML = True
if STOP_BEFORE_ML:
    print("Stopping here so you can verify forcings & var_wl before any modeling.")
    raise SystemExit(0)



# ================================================
# Colab: Seiche forecast with PatchTST (XGB feature pick, forced baro_var4h)
# ================================================

# -- If needed (Colab usually has torch already):
!pip -q install xgboost

import io, gzip, re, json, os, time, math, requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
from contextlib import nullcontext
from sklearn.metrics import r2_score
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# --------------------------------
# USER KNOBS
# --------------------------------
YEARS                = list(range(2008, 2026))
DT_MINUTES           = 6                     # base grid
HISTORY_HOURS        = 12
HORIZONS_MIN         = [60, 120, 240]
PRIMARY_HORIZON      = 60
MAX_GAP_HOURS        = 5
TOP_LAG_FEATURES     = 16                   # keep a bit larger; we’ll still force baro features

# WL bandpass + variance
VAR_WIN_HOURS        = 2.0                  # centered rolling variance window (hours)
WL_BAND_MIN_MIN      = 20.0                 # minutes
WL_BAND_MAX_MIN      = 30.0                 # minutes

# NOAA stations + ranges
NOAA_WIND_STATION    = "9435380"            # Newport / South Beach (CO-OPS)
NOAA_BEGIN_DATE      = "20080101"
NOAA_END_DATE        = "20251001"
NOAA_CHUNK_DAYS      = 365
NOAA_WL_STATION      = "9435380"
WL_BEGIN_DATE        = "20080101"
WL_END_DATE          = "20251001"

# NDBC buoy 46050 spectra/stdmet
BUOY                 = "46050"

# HMSC monthly archive
HMSC_BASE_URL        = "http://weather.hmsc.oregonstate.edu/weather/weatherproject/archive/{yyyy}/HMSC_{yyyymm}.dat"
HMSC_UTC_OFFSET_H    = +8

# Barometric pressure high-pass + variance
BARO_HP_CUTOFF_H     = 4.0                  # high-pass cutoff (hours)
BARO_VAR_WIN_H       = 2.0                  # centered rolling variance (hours)
FORCE_BARO_IN_XGB    = True                 # force include baro features
BARO_FORCE_LAGS_MIN  = [0, 240]             # lags to try/force (minutes)

# Earthquake events (UTC) — add/edit as needed
EQ_EVENTS = [
    ("2011-03-11 05:46:00", 9.1),   # Tohoku
    ("2015-09-16 22:54:00", 8.3),   # Illapel, Chile
]
# EQ amplitude ~10^(M-7); decay tau (hours) ≈ 6.67*M - 30 (M9 ≈ 30h, M7.5 ≈ 20h)
EQ_DECAY_TAU_FUNC = lambda M: max(8.0, 6.6667 * M - 30.0)
EQ_AMPLITUDE_FUNC = lambda M: 10.0**(M - 7.0)

# Spike emphasis (targets) for PatchTST training
WEIGHT_SPIKES       = True
SPIKE_P90           = 0.90
SPIKE_WEIGHT        = 2.0

# PatchTST model knobs
PATCH_LEN           = 24      # 24*6min = 144 min patches
PATCH_STRIDE        = 6       # slide by 36 min
D_MODEL             = 192
NHEAD               = 4
NLAYERS             = 3
FF_DIM              = 384
DROPOUT             = 0.15
LR                  = 2e-3
EPOCHS              = 20
BATCH_SIZE          = 512

# Plot window
PRED_SPLIT          = "test"  # "test" or "val"
PRED_START          = pd.Timestamp("2022-11-01 00:00:00")
PRED_END            = pd.Timestamp("2022-12-20 23:59:59")
PLOT_LAST_DAYS      = 40

# Time splits
TRAIN_END           = pd.Timestamp("2018-12-31 23:59:59")
VAL_END             = pd.Timestamp("2022-12-31 23:59:59")
MAX_SAMPLES_SPLIT   = 60_000

# --------------------------------
# HTTP session with retries
# --------------------------------
def _make_session():
    s = requests.Session()
    retry = Retry(total=4, backoff_factor=0.6, status_forcelist=(429,500,502,503,504),
                  allowed_methods=["HEAD","GET","OPTIONS"], raise_on_status=False)
    s.headers.update({"User-Agent":"Mozilla/5.0 (X11; Linux x86_64) SeichePatchTST/1.0", "Accept":"*/*"})
    s.mount("https://", HTTPAdapter(max_retries=retry))
    s.mount("http://", HTTPAdapter(max_retries=retry))
    return s
session = _make_session()

def _ndbc_to_float(tok:str) -> float:
    s = str(tok).strip()
    if s == "" or s.upper() == "MM": return np.nan
    if re.fullmatch(r"-?9+(\.0+)?", s): return np.nan
    try: return float(s)
    except: return np.nan

def _datestr(dt) -> str:
    return pd.Timestamp(dt).strftime("%Y%m%d")

# --------------------------------
# NDBC 46050 STDMET (Hs/DPD/APD/MWD) — we will EXCLUDE Hs later
# --------------------------------
STDMET_PHP_PAT = f"https://www.ndbc.noaa.gov/view_text_file.php?filename={BUOY}h"+"{year}.txt.gz&dir=data/historical/stdmet/"

def fetch_stdmet_year(year:int)->str|None:
    for url in (STDMET_PHP_PAT.format(year=year),):
        try:
            r = session.get(url, timeout=40)
            if r.status_code==200 and r.text and "Page Not Found" not in r.text:
                return r.text
        except: pass
    return None

def parse_stdmet_text(txt:str)->pd.DataFrame:
    rows=[]
    for ln in txt.splitlines():
        if not ln or ln.lstrip().startswith("#"): continue
        parts = re.split(r"\s+", ln.strip())
        if len(parts) < 6: continue
        y,mo,dy,hh,mi = parts[:5]
        try:
            y=int(y); mo=int(mo); dy=int(dy); hh=int(hh); mi=int(mi)
            year = 2000+y if y<100 else y
            ts = pd.Timestamp(year=year, month=mo, day=dy, hour=hh, minute=mi, tz="UTC").tz_localize(None)
        except: continue
        vals = parts[5:]
        if len(vals) < 13: vals = (vals + ["MM"]*13)[:13]
        vals = [_ndbc_to_float(v) for v in vals]
        rows.append([ts] + vals)
    if not rows: return pd.DataFrame()
    cols = ["time","WDIR","WSPD","GST","WVHT","DPD","APD","MWD","PRES","ATMP","WTMP","DEWP","VIS","TIDE"]
    df = pd.DataFrame(rows, columns=cols).sort_values("time").drop_duplicates("time")
    df = df.rename(columns={"WVHT":"Hs"}).reset_index(drop=True)
    for c,thr in [("Hs",50.0),("DPD",50.0),("APD",50.0)]:
        if c in df.columns: df.loc[df[c]>thr, c]=np.nan
    df.loc[(df["MWD"]<0)|(df["MWD"]>360),"MWD"]=np.nan
    keep = ["time","Hs","DPD","APD","MWD"]
    for k in keep:
        if k not in df.columns: df[k]=np.nan
    return df[keep]

def build_stdmet(years: List[int]) -> pd.DataFrame:
    frames=[]
    for y in years:
        txt = fetch_stdmet_year(y)
        if not txt: continue
        dfy = parse_stdmet_text(txt)
        if not dfy.empty: frames.append(dfy)
    if not frames: return pd.DataFrame(columns=["time","Hs","DPD","APD","MWD"])
    return pd.concat(frames, ignore_index=True).sort_values("time").drop_duplicates("time").reset_index(drop=True)

df_stdmet = build_stdmet(YEARS)

# --------------------------------
# NDBC 46050 SWDEN → spectral bands high_HIG (<0.05Hz), Hswell (0.05–0.14Hz), Hsea (0.14–0.30Hz)
# --------------------------------
SWDEN_PHP_PATS = [
    f"https://www.ndbc.noaa.gov/view_text_file.php?filename={BUOY}w"+"{year}.txt.gz&dir=data/historical/swden/",
    f"https://www.ndbc.noaa.gov/view_text_file.php?filename={BUOY}w"+"{year}.txt.gz&dir=data/swden/"+BUOY+"/",
]

def fetch_swden_year(year:int)->str|None:
    for url in SWDEN_PHP_PATS:
        try:
            r = session.get(url.format(year=year), timeout=40)
            if r.status_code==200 and r.text and "Page Not Found" not in r.text:
                return r.text
        except: pass
    return None

def split_swden_blocks(lines:List[str]):
    header=[]; i=0
    while i<len(lines) and lines[i].startswith("#"):
        header.append(lines[i]); i+=1
    while i<len(lines) and not lines[i].strip():
        i+=1
    freq_line = lines[i].rstrip("\n") if i<len(lines) else None
    i += 1 if i<len(lines) else 0
    data=[ln for ln in lines[i:] if ln.strip()]
    return header, freq_line, data

def parse_swden_year(txt:str)->pd.DataFrame:
    lines = txt.splitlines()
    _, freq_line, data_lines = split_swden_blocks(lines)
    if not freq_line: return pd.DataFrame()
    freq_tokens = [t for t in re.split(r"\s+", freq_line.strip()) if t]
    if any(k in freq_tokens[:6] for k in ("YY","MM","DD","hh")):
        idx = lines.index(freq_line)
        j=idx+1
        while j<len(lines) and not lines[j].strip(): j+=1
        if j<len(lines):
            freq_tokens = [t for t in re.split(r"\s+", lines[j].strip()) if t]
            data_lines  = [ln for ln in lines[j+1:] if ln.strip()]
        else:
            return pd.DataFrame()

    def _num(tok):
        try: return float(tok)
        except: return np.nan
    freqs = np.array([_num(t) for t in freq_tokens], dtype=float)
    freqs = freqs[np.isfinite(freqs)]
    if freqs.size==0: return pd.DataFrame()

    dfw = np.empty_like(freqs)
    if freqs.size==1:
        dfw[:] = 0.0
    else:
        dfw[1:-1] = 0.5*(freqs[2:] - freqs[:-2])
        dfw[0]    = freqs[1] - freqs[0]
        dfw[-1]   = freqs[-1] - freqs[-2]
        dfw = np.where(dfw<0, np.nan, dfw)

    mask_hig   = (freqs < 0.05)
    mask_swell = (freqs >= 0.05) & (freqs <= 0.14)
    mask_sea   = (freqs > 0.14) & (freqs <= 0.30)

    rows=[]
    for ln in data_lines:
        parts = re.split(r"\s+", ln.strip())
        if len(parts) < 4: continue
        y,mo,dy,hh = parts[:4]
        try:
            y=int(y); mo=int(mo); dy=int(dy); hh=int(hh)
            year = 2000+y if y<100 else y
            ts = pd.Timestamp(year=year, month=mo, day=dy, hour=hh, tz="UTC").tz_localize(None)
        except: continue

        vals = np.array([_ndbc_to_float(v) for v in parts[4:]], dtype=float)
        if vals.size != freqs.size:
            if vals.size < freqs.size: vals = np.pad(vals, (0, freqs.size - vals.size), constant_values=np.nan)
            else: vals = vals[:freqs.size]

        def m0(mask): return np.nansum(vals[mask] * dfw[mask]) if mask.any() else np.nan
        high_HIG = 4.0*np.sqrt(m0(mask_hig))   if mask_hig.any()   else np.nan
        Hswell   = 4.0*np.sqrt(m0(mask_swell)) if mask_swell.any() else np.nan
        Hsea     = 4.0*np.sqrt(m0(mask_sea))   if mask_sea.any()   else np.nan
        rows.append((ts, high_HIG, Hswell, Hsea))

    if not rows: return pd.DataFrame()
    return (pd.DataFrame(rows, columns=["time","high_HIG","Hswell","Hsea"])
            .sort_values("time").drop_duplicates("time").reset_index(drop=True))

def build_swden(years: List[int]) -> pd.DataFrame:
    frames=[]
    for y in years:
        txt = fetch_swden_year(y)
        if not txt: continue
        dfy = parse_swden_year(txt)
        if not dfy.empty: frames.append(dfy)
    if not frames: return pd.DataFrame(columns=["time","high_HIG","Hswell","Hsea"])
    return pd.concat(frames, ignore_index=True).sort_values("time").drop_duplicates("time").reset_index(drop=True)

df_swden = build_swden(YEARS)

# --------------------------------
# NOAA CO-OPS wind (hourly) → speed, dir (no u/v in features)
# --------------------------------
def parse_noaa_wind_csv(txt:str)->pd.DataFrame:
    df = pd.read_csv(io.StringIO(txt))
    tcol = [c for c in df.columns if str(c).strip().lower() in ("t","time","date time","date_time","date")]
    tcol = tcol[0] if tcol else df.columns[0]
    df["time"] = pd.to_datetime(df[tcol], errors="coerce", utc=True).dt.tz_localize(None)
    def _find(keys):
        keys = [k.lower() for k in keys]
        for c in df.columns:
            s = str(c).lower()
            if any(k == s or k in s for k in keys): return c
        return None
    sp_col = _find(["s","speed"])
    dir_col= _find(["d","dir","direction"])
    out = pd.DataFrame({"time":df["time"]})
    out["wind_speed"] = pd.to_numeric(df[sp_col], errors="coerce") if sp_col else np.nan
    out["wind_dir"]   = pd.to_numeric(df[dir_col], errors="coerce") if dir_col else np.nan
    out.loc[(out["wind_dir"]<0)|(out["wind_dir"]>360),"wind_dir"]=np.nan
    out.loc[(out["wind_speed"]<0)|(out["wind_speed"]>100),"wind_speed"]=np.nan
    return (out.sort_values("time").dropna(subset=["time"])
            .drop_duplicates("time").reset_index(drop=True))

def fetch_noaa_wind_dataframe(begin_date:str, end_date:str, station:str, days_per_chunk:int=365)->pd.DataFrame:
    start = pd.to_datetime(begin_date, format="%Y%m%d"); end = pd.to_datetime(end_date, format="%Y%m%d")
    chunks=[]; cur=start
    while cur <= end:
        chunk_end = min(end, cur + pd.Timedelta(days=days_per_chunk-1))
        chunks.append((cur, chunk_end)); cur = chunk_end + pd.Timedelta(days=1)
    frames=[]
    for a,b in chunks:
        url = ("https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
               f"?begin_date={_datestr(a)}&end_date={_datestr(b)}"
               f"&station={station}&product=wind&time_zone=gmt&interval=h&units=metric"
               "&application=SeichePatchTST&format=csv")
        try:
            r = session.get(url, timeout=60)
            if r.status_code==200 and r.content:
                dfc = parse_noaa_wind_csv(r.content.decode("utf-8","ignore"))
                if not dfc.empty: frames.append(dfc)
        except: pass
    if not frames: return pd.DataFrame(columns=["time","wind_speed","wind_dir"])
    return pd.concat(frames, ignore_index=True).sort_values("time").drop_duplicates("time").reset_index(drop=True)

df_noaa_wind = fetch_noaa_wind_dataframe(NOAA_BEGIN_DATE, NOAA_END_DATE, NOAA_WIND_STATION, days_per_chunk=NOAA_CHUNK_DAYS)

# ================= NOAA + HMSC baro merge rules =================
# - NOAA air_pressure (hourly, metric) -> hPa -> Pa
# - HMSC AvgBP (inHg, 5-min) -> Pa
# - Fill NOAA gaps:
#     * runs >= 3 hours: fill from HMSC (on hourly grid)
#     * runs < 3 hours: spline interpolation (NOAA only)
# - Then interpolate merged Pa to t_grid and compute rolling variance

INHG_TO_PA = 3386.389  # 1 inch Hg ≈ 3386.389 Pa

# --- 1) Ensure df_air from NOAA exists (hourly), convert to Pa ---
if 'df_air' not in globals():
    raise RuntimeError("df_air (NOAA air pressure) is not defined. Fetch it first.")
df_air = df_air.dropna(subset=["time"]).drop_duplicates("time").sort_values("time").reset_index(drop=True)
df_air["air_pressure_pa"] = (df_air["air_pressure_hpa"].astype(float) * 100.0).astype("float64")

# Hourly base grid for merging/filling
ap_start = df_air["time"].min()
ap_end   = df_air["time"].max()
hourly_grid = pd.date_range(ap_start.floor("h"), ap_end.ceil("h"), freq="h")

s_noaa_pa = (df_air.set_index("time")["air_pressure_pa"]
             .reindex(hourly_grid)  # align to hourly
             .astype("float64"))

# --- 2) Build HMSC AvgBP (inHg) -> Pa over same window ---
def fetch_hmsc_month_text(year:int, month:int) -> str|None:
    url = f"http://weather.hmsc.oregonstate.edu/weather/weatherproject/archive/{year}/HMSC_{year}{month:02d}.dat"
    try:
        r = session.get(url, timeout=60)
        if r.status_code==200 and r.content:
            return r.content.decode("utf-8", "ignore")
    except Exception:
        pass
    return None

def parse_hmsc_dat_avgbp_pa(txt:str) -> pd.DataFrame:
    # Find the header line that starts the CSV (TIMESTAMP,...)
    lines = txt.splitlines()
    start_idx = None
    for i,ln in enumerate(lines):
        s = ln.strip().strip('"')
        if s.startswith("TIMESTAMP"):
            start_idx = i
            break
    if start_idx is None:
        return pd.DataFrame(columns=["time","bp_pa"])

    content = "\n".join(lines[start_idx:])
    df = pd.read_csv(io.StringIO(content))
    # Time
    t = pd.to_datetime(df.iloc[:,0], errors="coerce")
    # Treat timestamps as America/Los_Angeles local, convert to UTC naive for merging with NOAA GMT
    try:
        t = t.dt.tz_localize("America/Los_Angeles", nonexistent="shift_forward", ambiguous="NaT") \
             .dt.tz_convert("UTC").dt.tz_localize(None)
    except Exception:
        # Fallback: assume already UTC if tz db not available
        pass

    df.rename(columns={df.columns[0]:"TIMESTAMP"}, inplace=True)

    # Find AvgBP column (case-insensitive match)
    target = None
    for c in df.columns:
        if "avgbp" in str(c).strip().lower():
            target = c; break
    if target is None:
        return pd.DataFrame(columns=["time","bp_pa"])

    avgbp_inhg = pd.to_numeric(df[target], errors="coerce").astype("float64")
    # sanity: realistic sea-level pressure in inches Hg (~27–32)
    avgbp_inhg = avgbp_inhg.where(avgbp_inhg.between(26.0, 33, inclusive="both"))

    out = pd.DataFrame({"time": t, "bp_pa": avgbp_inhg * INHG_TO_PA})
    out = (out.dropna(subset=["time"])
              .sort_values("time")
              .drop_duplicates("time")
              .reset_index(drop=True))
    return out

def month_iter(start:pd.Timestamp, end:pd.Timestamp):
    y,m = start.year, start.month
    while pd.Timestamp(year=y, month=m, day=1) <= end:
        yield (y,m)
        if m==12: y,m=y+1,1
        else: m+=1

# Fetch months that cover NOAA window (pad one month on each side)
start_pad = (ap_start - pd.Timedelta(days=31)).normalize()
end_pad   = (ap_end   + pd.Timedelta(days=31)).normalize()

hmsc_frames = []
for y,m in month_iter(start_pad, end_pad):
    txt = fetch_hmsc_month_text(y,m)
    if not txt: continue
    dfm = parse_hmsc_dat_avgbp_pa(txt)
    if not dfm.empty:
        # keep only within padded window to reduce size
        mask = (dfm["time"] >= start_pad) & (dfm["time"] <= end_pad)
        hmsc_frames.append(dfm.loc[mask])

df_hmsc_bp = (pd.concat(hmsc_frames, ignore_index=True)
              .sort_values("time")
              .drop_duplicates("time")
              .reset_index(drop=True)) if hmsc_frames else pd.DataFrame(columns=["time","bp_pa"])

# Resample HMSC bp to hourly with time interpolation (so we can drop into NOAA hourly gaps)
if len(df_hmsc_bp):
    s_hmsc_pa_hourly = (df_hmsc_bp.set_index("time")["bp_pa"]
                        .reindex(hourly_grid)
                        .interpolate(method="time", limit_area="inside")
                        .astype("float64"))
else:
    s_hmsc_pa_hourly = pd.Series(index=hourly_grid, dtype="float64")

# --- 3) Fill rules on hourly grid ---
filled = s_noaa_pa.copy()

# Identify contiguous NaN runs
nan_mask = filled.isna().to_numpy()
if nan_mask.any():
    # group NaN runs
    idx = np.arange(len(nan_mask))
    # start/end indices of each NaN block
    starts = np.where(~nan_mask[:-1] & nan_mask[1:])[0] + 1 if len(nan_mask) > 1 else np.array([], dtype=int)
    ends   = np.where(nan_mask[:-1] & ~nan_mask[1:])[0] + 1 if len(nan_mask) > 1 else np.array([], dtype=int)
    # edge cases
    if nan_mask[0]:  starts = np.r_[0, starts]
    if nan_mask[-1]: ends   = np.r_[ends, len(nan_mask)]

    for s,e in zip(starts, ends):
        run_len_hours = e - s  # hourly freq
        if run_len_hours >= 3:
            # >= 3h: fill from HMSC hourly if available
            seg_idx = hourly_grid[s:e]
            hmsc_vals = s_hmsc_pa_hourly.loc[seg_idx]
            filled.iloc[s:e] = hmsc_vals.values
    # For any remaining NaNs (i.e., runs < 3h or where HMSC missing), do spline on NOAA only
    if filled.isna().any():
        # spline requires some finite points; fall back to 'time' if sparse
        try:
            filled = filled.interpolate(method="spline", order=3, limit_area="inside")
        except Exception:
            filled = filled.interpolate(method="time", limit_area="inside")

# --- 4) Push merged Pa to your 6-min model grid and compute rolling variance ---
def _interp_to_grid_series(times: pd.DatetimeIndex, values: np.ndarray, grid: pd.DatetimeIndex) -> np.ndarray:
    s = pd.Series(values, index=times)
    u = s.reindex(pd.to_datetime(sorted(set(s.index).union(set(grid)))))
    u = u.interpolate(method="time", limit_area="inside")
    return u.reindex(pd.to_datetime(grid)).to_numpy()

if 't_grid' not in globals():
    if 'df_varWL' in globals() and len(df_varWL):
        t_grid = pd.date_range(df_varWL["time"].min(), df_varWL["time"].max(), freq=f"{DT_MINUTES}min")
    else:
        raise RuntimeError("t_grid is required to map air pressure to model grid.")

air_pa_on_grid = _interp_to_grid_series(hourly_grid, filled.to_numpy(dtype="float64"), t_grid)

# centered rolling variance in Pa^2
win = max(3, int(round((BARO_VAR_WIN_H*60.0)/DT_MINUTES)))  # BARO_VAR_WIN_H hours
minp = max(2, int(0.8*win))
baro_var = (pd.Series(air_pa_on_grid, index=t_grid)
            .rolling(window=win, center=True, min_periods=minp).var()
            .astype("float32"))

# --- 5) Inject into seq_mat and df_all ---
if 'seq_mat' not in globals():
    seq_mat = {}
seq_mat["air_pressure_pa"] = air_pa_on_grid.astype("float32")
seq_mat["baro_var4h"]      = baro_var.to_numpy(dtype="float32")  # keep name for downstream code

# Merge into df_all
base = pd.DataFrame({"time": t_grid, "air_pressure_pa": seq_mat["air_pressure_pa"], "baro_var4h": seq_mat["baro_var4h"]})
if 'df_all' in globals() and len(df_all):
    df_all = df_all.merge(base, on="time", how="left")
else:
    df_all = base.copy()

# Quick diagnostics
print("Air pressure (Pa) non-NaN on t_grid:", np.isfinite(seq_mat["air_pressure_pa"]).sum(), "/", len(t_grid))
print("baro_var4h non-NaN on t_grid:", np.isfinite(seq_mat["baro_var4h"]).sum(), "/", len(t_grid))

# --------------------------------
# WL (6-min) → bandpass(20–30 min) → centered variance (2 h)
# --------------------------------
from scipy.signal import butter, filtfilt

def parse_noaa_wl_csv(txt:str)->pd.DataFrame:
    if txt.strip().lower().startswith("error"):
        return pd.DataFrame(columns=["time","wl"])
    df = pd.read_csv(io.StringIO(txt))
    if df.shape[1] < 2: return pd.DataFrame(columns=["time","wl"])
    df.columns = [str(c).strip().lower().replace("  "," ") for c in df.columns]
    tcol=None
    for k in ("date time","date_time","time","t","date"):
        if k in df.columns: tcol=k; break
    if tcol is None: return pd.DataFrame(columns=["time","wl"])
    df["time"] = pd.to_datetime(df[tcol], errors="coerce", utc=True).dt.tz_localize(None)
    vcol=None
    for k in ("water level","water_level","wl","v"):
        if k in df.columns: vcol=k; break
    if vcol is None: vcol = df.columns[1]
    df["wl"] = pd.to_numeric(df[vcol], errors="coerce")
    out = df[["time","wl"]].dropna(subset=["time"]).sort_values("time").drop_duplicates("time")
    out.loc[(out["wl"]<-50)|(out["wl"]>50),"wl"]=np.nan
    if out["wl"].notna().sum()==0: return pd.DataFrame(columns=["time","wl"])
    return out.reset_index(drop=True)

def fetch_noaa_wl(begin_date:str, end_date:str, station:str, days_per_chunk:int=31)->pd.DataFrame:
    start = pd.to_datetime(begin_date, format="%Y%m%d"); end = pd.to_datetime(end_date, format="%Y%m%d")
    chunks=[]; cur=start
    while cur <= end:
        chunk_end = min(end, cur + pd.Timedelta(days=days_per_chunk-1))
        chunks.append((cur, chunk_end)); cur = chunk_end + pd.Timedelta(days=1)
    frames=[]
    for a,b in chunks:
        url = ("https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
               f"?begin_date={_datestr(a)}&end_date={_datestr(b)}"
               f"&station={station}&product=water_level&datum=MLLW&time_zone=gmt&units=metric"
               f"&interval=6&application=SeichePatchTST&format=csv")
        try:
            r = session.get(url, timeout=90)
            if r.status_code==200 and r.content:
                dfc = parse_noaa_wl_csv(r.content.decode("utf-8","ignore"))
                if not dfc.empty: frames.append(dfc)
        except: pass
    if not frames: return pd.DataFrame(columns=["time","wl"])
    return pd.concat(frames, ignore_index=True).sort_values("time").drop_duplicates("time").reset_index(drop=True)

def bandpass_then_var(series:pd.DataFrame, val_col:str, dt_minutes:int,
                      band_min_min:float, band_max_min:float, var_win_h:float)->pd.DataFrame:
    if series.empty: return pd.DataFrame(columns=["time","var"])
    t0,t1 = series["time"].min(), series["time"].max()
    grid = pd.date_range(t0.floor(f"{dt_minutes}min"), t1.ceil(f"{dt_minutes}min"), freq=f"{dt_minutes}min")
    s = series.set_index("time")[val_col].reindex(grid)
    s = s.interpolate(method="time", limit_area="inside")
    dt_sec = dt_minutes*60.0; fs=1.0/dt_sec
    f_lo = 1.0/(band_max_min*60.0)
    f_hi = 1.0/(band_min_min*60.0)
    wn = [max(1e-6, f_lo/(fs/2.0)), min(0.999, f_hi/(fs/2.0))]
    b,a = butter(6, wn, btype="bandpass")
    x = s.to_numpy(dtype=float)
    if np.isnan(x).any():
        idx = np.arange(len(x)); good = np.isfinite(x); x[~good] = np.interp(idx[~good], idx[good], x[good])
    x_bp = filtfilt(b,a,x,method="gust")
    win = max(3, int(round((var_win_h*60.0)/dt_minutes))); minp = max(2, int(0.8*win))
    var_series = pd.Series(x_bp, index=grid).rolling(window=win, center=True, min_periods=minp).var()
    return pd.DataFrame({"time":grid, "var":var_series.astype("float32")}).dropna(subset=["time"]).reset_index(drop=True)

def highpass_then_var(series:pd.DataFrame, val_col:str, dt_minutes:int,
                      hp_cutoff_h:float, var_win_h:float)->pd.DataFrame:
    """1st: high-pass (remove periods longer than hp_cutoff_h), then centered rolling variance."""
    if series.empty: return pd.DataFrame(columns=["time","var"])
    t0,t1 = series["time"].min(), series["time"].max()
    grid = pd.date_range(t0.floor(f"{dt_minutes}min"), t1.ceil(f"{dt_minutes}min"), freq=f"{dt_minutes}min")
    s = series.set_index("time")[val_col].reindex(grid)
    s = s.interpolate(method="time", limit_area="inside")
    dt_sec = dt_minutes*60.0; fs=1.0/dt_sec
    f_c = 1.0/(hp_cutoff_h*3600.0)  # Hz
    wn = max(1e-6, min(0.999, f_c/(fs/2.0)))
    b,a = butter(4, wn, btype="highpass")   # 4th order is plenty here
    x = s.to_numpy(dtype=float)
    if np.isnan(x).any():
        idx = np.arange(len(x)); good = np.isfinite(x); x[~good] = np.interp(idx[~good], idx[good], x[good])
    x_hp = filtfilt(b,a,x,method="gust")
    win = max(3, int(round((var_win_h*60.0)/dt_minutes))); minp = max(2, int(0.8*win))
    var_series = pd.Series(x_hp, index=grid).rolling(window=win, center=True, min_periods=minp).var()
    return pd.DataFrame({"time":grid, "var":var_series.astype("float32")}).dropna(subset=["time"]).reset_index(drop=True)

# --------------------------------
# Build all sources
# --------------------------------
df_wl6   = fetch_noaa_wl(WL_BEGIN_DATE, WL_END_DATE, NOAA_WL_STATION, days_per_chunk=31)
df_varWL = bandpass_then_var(df_wl6, "wl", DT_MINUTES, WL_BAND_MIN_MIN, WL_BAND_MAX_MIN, VAR_WIN_HOURS)

df_46050 = df_stdmet.copy()
df_swd   = df_swden.copy()

# Wind: choose coverage window from wind if present, else WL
if len(df_noaa_wind):
    fill_start, fill_end = df_noaa_wind["time"].min(), df_noaa_wind["time"].max()
else:
    candidates=[]
    if len(df_swd): candidates.append((df_swd["time"].min(), df_swd["time"].max()))
    if len(df_46050): candidates.append((df_46050["time"].min(), df_46050["time"].max()))
    if len(df_wl6): candidates.append((df_wl6["time"].min(), df_wl6["time"].max()))
    if candidates:
        fill_start = min(a for a,_ in candidates); fill_end = max(b for _,b in candidates)
    else:
        fill_start = pd.Timestamp("2008-01-01"); fill_end=pd.Timestamp("2025-10-01")

# HMSC wind/dir/baro hourlies
df_hmsc = fetch_hmsc_range(fill_start, fill_end)

# Build unified hourly grid for wind fill
hourly_grid = pd.date_range(fill_start.floor("h"), fill_end.ceil("h"), freq="h")
noaa = (df_noaa_wind.set_index("time").reindex(hourly_grid).rename_axis("time").reset_index()) if len(df_noaa_wind) else pd.DataFrame({"time":hourly_grid})
noaa.rename(columns={"wind_speed":"speed","wind_dir":"dir"}, inplace=True)

if len(df_hmsc):
    hmsc = (df_hmsc.set_index("time").reindex(hourly_grid).rename_axis("time").reset_index())
else:
    hmsc = pd.DataFrame({"time":hourly_grid, "hmsc_speed_ms":np.nan,"hmsc_dir_deg":np.nan,"hmsc_baro_hpa":np.nan})

# fill wind speed/dir gaps from HMSC if both present
gap_mask = noaa[["speed","dir"]].isna().any(axis=1) if set(["speed","dir"]).issubset(noaa.columns) else pd.Series(True, index=noaa.index)
fillable = gap_mask & hmsc[["hmsc_speed_ms","hmsc_dir_deg"]].notna().all(axis=1)
noaa.loc[fillable, "speed"] = hmsc.loc[fillable, "hmsc_speed_ms"].values
noaa.loc[fillable, "dir"]   = hmsc.loc[fillable, "hmsc_dir_deg"].values

df_wind = pd.DataFrame({
    "time": noaa["time"],
    "wind9435380_speed": noaa.get("speed", pd.Series(index=noaa.index, dtype=float)).astype("float32"),
    "wind9435380_dir":   noaa.get("dir",   pd.Series(index=noaa.index, dtype=float)).astype("float32"),
}).dropna(subset=["time"]).reset_index(drop=True)

# Baro (hourly) from HMSC if present → highpass + variance
if "hmsc_baro_hpa" in hmsc.columns and hmsc["hmsc_baro_hpa"].notna().any():
    df_baro = hmsc[["time","hmsc_baro_hpa"]].copy()
    df_baro = df_baro.dropna(subset=["time"]).drop_duplicates("time").sort_values("time").reset_index(drop=True)
    df_baro_var = highpass_then_var(df_baro, "hmsc_baro_hpa", 60, BARO_HP_CUTOFF_H, BARO_VAR_WIN_H)
else:
    df_baro = pd.DataFrame(columns=["time","hmsc_baro_hpa"])
    df_baro_var = pd.DataFrame(columns=["time","var"])

# --------------------------------
# Build common DT grid + assemble inputs
# --------------------------------
def interp_timegrid(t_src:pd.Series, x_src:np.ndarray, t_grid:pd.DatetimeIndex)->np.ndarray:
    s = pd.Series(x_src, index=pd.to_datetime(t_src))
    u = s.reindex(pd.to_datetime(sorted(set(s.index).union(set(t_grid)))))
    u = u.interpolate(method="time", limit_area="inside")
    return u.reindex(pd.to_datetime(t_grid)).to_numpy()

# overlap
t0_candidates = [df_varWL["time"].min()] if len(df_varWL) else []
t1_candidates = [df_varWL["time"].max()] if len(df_varWL) else []
for d, col in [(df_swd,"time"), (df_46050,"time"), (df_wind,"time"), (df_baro,"time"), (df_baro_var,"time")]:
    if len(d):
        t0_candidates.append(d[col].min()); t1_candidates.append(d[col].max())
t0 = max(t0_candidates); t1 = min(t1_candidates)
t_grid = pd.date_range(t0, t1, freq=f"{DT_MINUTES}min")

# target
var_on_grid = interp_timegrid(df_varWL["time"], df_varWL["var"].values, t_grid)

# inputs
seq_mat = {}
if len(df_swd):
    seq_mat["high_HIG"] = interp_timegrid(df_swd["time"], df_swd["high_HIG"].values, t_grid)
    seq_mat["Hswell"]   = interp_timegrid(df_swd["time"], df_swd["Hswell"].values, t_grid)
    seq_mat["Hsea"]     = interp_timegrid(df_swd["time"], df_swd["Hsea"].values, t_grid)

for c in ["DPD","APD","MWD"]:  # EXCLUDE Hs
    if len(df_46050):
        seq_mat[c] = interp_timegrid(df_46050["time"], df_46050[c].values, t_grid)

if len(df_wind):
    seq_mat["wind9435380_speed"] = interp_timegrid(df_wind["time"], df_wind["wind9435380_speed"].values, t_grid)
    seq_mat["wind9435380_dir"]   = interp_timegrid(df_wind["time"], df_wind["wind9435380_dir"].values,   t_grid)

if len(df_baro):
    seq_mat["hmsc_baro_hpa"] = interp_timegrid(df_baro["time"], df_baro["hmsc_baro_hpa"].values, t_grid)
if len(df_baro_var):
    seq_mat["baro_var4h"] = interp_timegrid(df_baro_var["time"], df_baro_var["var"].values, t_grid)
else:
    seq_mat["baro_var4h"] = np.full(len(t_grid), np.nan, dtype=float)

# EQ_decay
def build_eq_decay_series(t_grid: pd.DatetimeIndex, eq_events: list[tuple[str|pd.Timestamp, float]]) -> np.ndarray:
    if not eq_events or len(t_grid)==0:
        return np.zeros(len(t_grid), dtype=float)
    out = np.zeros(len(t_grid), dtype=float)
    for t0_evt, M in eq_events:
        t0_evt = pd.Timestamp(t0_evt)
        amp = float(EQ_AMPLITUDE_FUNC(float(M)))
        tau_h = float(EQ_DECAY_TAU_FUNC(float(M))); tau_s = max(1.0, tau_h*3600.0)
        dt = (t_grid - t0_evt).total_seconds().astype(float)
        m = dt >= 0.0
        out[m] += amp * np.exp(-dt[m] / tau_s)
    return out

seq_mat["EQ_decay"] = build_eq_decay_series(t_grid, EQ_EVENTS)

# Unified gap mask (use the major channels)
def gap_intervals(t:pd.Series, x:np.ndarray, min_gap_hours:float)->List[Tuple[pd.Timestamp,pd.Timestamp]]:
    t = pd.to_datetime(t)
    if len(t) < 2: return []
    dt = np.diff(t.values).astype("timedelta64[s]").astype(int)
    jumps = np.where(dt > min_gap_hours*3600)[0]
    iv = [(t[j], t[j+1]) for j in jumps]
    s = pd.Series(x, index=t)
    isn = s.isna().to_numpy()
    if isn.any():
        starts = np.where(np.diff(np.r_[False, isn])==1)[0]
        ends   = np.where(np.diff(np.r_[isn, False])==-1)[0]
        for a,b in zip(starts, ends):
            if (t[b]-t[a]) >= pd.Timedelta(hours=min_gap_hours):
                iv.append((t[a], t[b]))
    return iv

def merge_intervals(intervals):
    if not intervals: return []
    z = sorted(intervals, key=lambda k: k[0]); out = [list(z[0])]
    for s,e in z[1:]:
        if s <= out[-1][1]: out[-1][1] = max(out[-1][1], e)
        else: out.append([s,e])
    return [(pd.to_datetime(a), pd.to_datetime(b)) for a,b in out]

all_intervals=[]
if len(df_swd):
    for c in ["high_HIG","Hswell","Hsea"]:
        all_intervals += gap_intervals(df_swd["time"], df_swd[c].values, MAX_GAP_HOURS)
if len(df_46050):
    for c in ["DPD","APD","MWD"]:
        all_intervals += gap_intervals(df_46050["time"], df_46050[c].values, MAX_GAP_HOURS)
if len(df_wind):
    for c in ["wind9435380_speed","wind9435380_dir"]:
        all_intervals += gap_intervals(df_wind["time"], df_wind[c].values, MAX_GAP_HOURS)
if len(df_baro):
    all_intervals += gap_intervals(df_baro["time"], df_baro["hmsc_baro_hpa"].values, MAX_GAP_HOURS)

merged_gaps = merge_intervals(all_intervals)
gap_mask = np.zeros(len(t_grid), dtype=bool)
for s,e in merged_gaps:
    gap_mask |= (t_grid>=s) & (t_grid<=e)

# Assemble df_all (target + inputs)
df_all = pd.DataFrame({"time": t_grid, "var1h": var_on_grid})
for c,v in seq_mat.items():
    df_all[c] = v
df_all = df_all.loc[~gap_mask].reset_index(drop=True)

# Post-align cleaning for stdmet heads
for col, thresh in [("DPD",50.0),("APD",50.0)]:
    if col in df_all.columns:
        m = df_all[col] > thresh
        if m.any(): df_all.loc[m, col] = np.nan
        df_all[col] = pd.Series(df_all[col].values, index=df_all["time"]).interpolate("time", limit_area="inside").values

# --------------------------------
# Lag features + XGB pick (force baro)
# --------------------------------
NON_EQ_MAX_LAG_MIN  = 4 * 60
EQ_MAX_LAG_MIN      = 24 * 60

def is_eq_base(name: str) -> bool:
    n = name.lower()
    return n.startswith("eq") or ("eq_" in n)

def build_lag_candidates(df: pd.DataFrame, bases: List[str], dt_minutes: int,
                         step_minutes: int = 30) -> Tuple[pd.DataFrame, List[str], Dict[str, str], Dict[str, int]]:
    lag_cols, series = [], {}
    base_map, lag_minutes_map = {}, {}
    for b in bases:
        x = df[b].astype("float32").to_numpy()
        if not np.isfinite(x).any(): continue
        max_lag_min = EQ_MAX_LAG_MIN if is_eq_base(b) else NON_EQ_MAX_LAG_MIN
        grid_min = list(range(0, max_lag_min + 1, int(step_minutes)))
        for L in grid_min:
            steps = int(round(L / dt_minutes))
            nm = f"{b}__lag_{L}min"
            lag_minutes_map[nm] = L; base_map[nm] = b
            if steps == 0:
                series[nm] = x
            else:
                s = pd.Series(x); series[nm] = s.shift(steps).to_numpy(dtype="float32")
            lag_cols.append(nm)
    lag_df = pd.DataFrame(series).reindex(range(len(df))).reset_index(drop=True)
    return lag_df, lag_cols, base_map, lag_minutes_map

# Exclude Hs explicitly; include everything else numeric
exclude_cols = set(["time","var1h","Hs"])  # drop Hs — redundant with spectral bands
cand_bases = [c for c in df_all.columns if (c not in exclude_cols and df_all[c].dtype != object)]

lag_df, lag_cols, base_map, lag_minutes_map = build_lag_candidates(df_all, cand_bases, DT_MINUTES)

# XGB rank one best per base
from xgboost import XGBRegressor
h_steps = max(1, int(round(PRIMARY_HORIZON / DT_MINUTES)))
tgt = df_all["var1h"].shift(-h_steps).reset_index(drop=True).astype("float32")
rank_data = pd.concat([lag_df, tgt.rename("target")], axis=1).dropna().reset_index(drop=True)
if rank_data.empty: raise RuntimeError("No data for XGB ranking; check coverage.")

X_rank = rank_data[lag_cols].to_numpy(dtype=np.float32)
y_rank = rank_data["target"].to_numpy(dtype=np.float32)

# existing code
X_rank = rank_data[lag_cols].to_numpy(dtype=np.float32)
y_rank = rank_data["target"].to_numpy(dtype=np.float32)

# === start Optuna integration ===
import optuna
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

# split ranking data (80/20) for tuning
split_idx = max(1, int(0.8 * len(X_rank)))
X_train, X_val = X_rank[:split_idx], X_rank[split_idx:]
y_train, y_val = y_rank[:split_idx], y_rank[split_idx:]

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 8),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 10.0),
    }
    tree_method = "gpu_hist" if torch.cuda.is_available() else "hist"
    model = XGBRegressor(
        objective="reg:squarederror",
        tree_method=tree_method,
        random_state=42,
        n_jobs=-1,
        **params
    )
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
              eval_metric="rmse", verbose=False, early_stopping_rounds=50)
    preds = model.predict(X_val)
    return mean_squared_error(y_val, preds)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

best_params = study.best_params
best_params.update({"objective": "reg:squarederror",
                    "tree_method": ("gpu_hist" if torch.cuda.is_available() else "hist"),
                    "random_state": 42,
                    "n_jobs": -1})
xgb_opt = XGBRegressor(**best_params)
xgb_opt.fit(X_rank, y_rank, verbose=False)

importances = dict(zip(lag_cols, xgb_opt.feature_importances_))

best_lag_per_base = {}
for nm, imp in importances.items():
    b = base_map[nm]
    prev = best_lag_per_base.get(b)
    if (prev is None) or (imp > prev[1]): best_lag_per_base[b] = (nm, float(imp))
best_list = [(b, nm, imp) for b,(nm,imp) in best_lag_per_base.items()]
best_list.sort(key=lambda t: t[2], reverse=True)
best_list = best_list[:TOP_LAG_FEATURES]

selected_lag_cols = [nm for _, nm, _ in best_list]
selected_bases    = [b  for b, _, _ in best_list]

# Force include baro features (and EQ if you want to guarantee)
def _prefer_lags(base_name:str, pref_lags:list[int])->str|None:
    for L in pref_lags:
        nm = f"{base_name}__lag_{L}min"
        if nm in lag_cols: return nm
    cands = [nm for nm in lag_cols if base_map.get(nm)==base_name]
    if not cands: return None
    return max(cands, key=lambda z: importances.get(z, 0.0))

for base in ["baro_var4h", "hmsc_baro_hpa", "EQ_decay"]:
    nm = _prefer_lags(base, BARO_FORCE_LAGS_MIN if base!="EQ_decay" else [0,240])
    if nm and nm not in selected_lag_cols:
        selected_lag_cols.append(nm); selected_bases.append(base)

print("\nSelected ONE best lag per base (forced baro/EQ included if available):")
for nm in selected_lag_cols:
    b = base_map[nm]
    imp = importances.get(nm, 0.0)
    print(f"  {nm:<32s} base={b:<16s} imp={imp:.6f} lag={lag_minutes_map[nm]}")

# Assemble minimal lag table
df_lag_all = pd.concat(
    [df_all[["time", "var1h"]].reset_index(drop=True),
     lag_df[selected_lag_cols].reset_index(drop=True)],
    axis=1
)

# --------------------------------
# PatchTST model & training
# --------------------------------
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

use_gpu = torch.cuda.is_available()
if use_gpu:
    torch.backends.cudnn.benchmark = True
    if hasattr(torch, "set_float32_matmul_precision"):
        torch.set_float32_matmul_precision("high")
def _amp_ctx(): return nullcontext()

class PatchEmbedding(nn.Module):
    def __init__(self, in_features:int, patch_len:int, stride:int, d_model:int):
        super().__init__()
        self.patch_len = patch_len
        self.stride    = stride
        self.proj      = nn.Linear(in_features*patch_len, d_model)
    def forward(self, x):
        B,T,F = x.shape
        N = 1 + (T - self.patch_len)//self.stride
        patches = []
        for i in range(N):
            s = i*self.stride; e = s + self.patch_len
            seg = x[:, s:e, :].reshape(B, -1)
            patches.append(seg)
        P = torch.stack(patches, dim=1)  # (B,N,F*patch_len)
        return self.proj(P)              # (B,N,d)

class PatchTST(nn.Module):
    def __init__(self, in_features, patch_len, stride, d_model=128, nhead=4, nlayers=3, ff_dim=256, drop=0.2):
        super().__init__()
        self.embed = PatchEmbedding(in_features, patch_len, stride, d_model)
        layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=ff_dim,
                                           dropout=drop, batch_first=True, norm_first=True)
        self.enc  = nn.TransformerEncoder(layer, num_layers=nlayers)
        self.head = nn.Sequential(nn.Linear(d_model, d_model//2), nn.ReLU(), nn.Linear(d_model//2, 1))
    def forward(self, x):
        z = self.embed(x); z = self.enc(z)
        return self.head(z[:, -1, :]).squeeze(-1)

def rmse(a,b): return float(np.sqrt(np.mean((np.asarray(a)-np.asarray(b))**2)))

# scaling helpers
def _safe_robust_params(X: np.ndarray, q_low=25.0, q_high=75.0, eps=1e-6):
    Xf = X.reshape(-1, X.shape[2])
    med = np.nanmedian(Xf, axis=0)
    q1  = np.nanpercentile(Xf, q_low, axis=0)
    q3  = np.nanpercentile(Xf, q_high, axis=0)
    iqr = q3 - q1
    scale = np.where(iqr <= eps, 1.0, iqr)
    return med.astype(np.float32), scale.astype(np.float32)

def fit_scaler_from_windows_safe(X_tr, X_va=None, X_te=None):
    for X in (X_tr, X_va, X_te):
        if X is not None and len(X) > 0:
            med, scale = _safe_robust_params(X)
            return {"center": med, "scale": scale}
    F = X_tr.shape[2] if (X_tr is not None and X_tr.size) else (
        X_va.shape[2] if (X_va is not None and X_va.size) else (
        X_te.shape[2] if (X_te is not None and X_te.size) else 1))
    return {"center": np.zeros(F, np.float32), "scale": np.ones(F, np.float32)}

def norm_windows(X, rs, clip=20.0):
    if len(X) == 0: return X
    F = X.shape[2]
    C = rs["center"].reshape(1, 1, F)
    S = rs["scale" ].reshape(1, 1, F)
    Z = (X - C) / S
    Z = np.nan_to_num(Z, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    if clip is not None: Z = np.clip(Z, -clip, clip, out=Z)
    return Z

def make_windows_from_lagged(df:pd.DataFrame, lag_cols:List[str],
                             history_hours:int, horizon_min:int, dt_minutes:int,
                             stride:int=1):
    steps_hist  = int(round(history_hours*60/dt_minutes))
    steps_ahead = int(round(horizon_min/dt_minutes))
    tab = df[["time","var1h"] + lag_cols].copy()
    tab = tab.dropna(subset=lag_cols + ["var1h"]).reset_index(drop=True)
    if len(tab) < steps_hist + steps_ahead + 1:
        return np.empty((0,steps_hist,len(lag_cols))), np.empty((0,)), pd.DatetimeIndex([])
    X_list, y_list, t_list = [], [], []
    F = len(lag_cols); Ttot = len(tab)
    for end_idx in range(steps_hist, Ttot - steps_ahead + 1, stride):
        s = end_idx - steps_hist
        k = end_idx - 1 + steps_ahead
        xb = tab.iloc[s:end_idx][lag_cols].values
        yv = tab.iloc[k]["var1h"]
        if np.any(~np.isfinite(xb)) or not np.isfinite(yv): continue
        X_list.append(xb.astype(np.float32))
        y_list.append(float(yv))
        t_list.append(pd.to_datetime(tab.iloc[k]["time"]))
    if not X_list:
        return np.empty((0,steps_hist,F)), np.empty((0,)), pd.DatetimeIndex([])
    return np.stack(X_list), np.array(y_list, dtype=np.float32), pd.DatetimeIndex(t_list)

def adaptive_index_split(X, y, t_idx, train_frac=0.70, val_frac=0.15):
    n = len(X)
    if n < 10:
        return (X[:0], y[:0], t_idx[:0]), (X[:0], y[:0], t_idx[:0]), (X, y, t_idx)
    a = max(1, int(round(train_frac * n)))
    b = max(a+1, int(round((train_frac + val_frac) * n)))
    b = min(b, n-1)
    return (X[:a], y[:a], t_idx[:a]), (X[a:b], y[a:b], t_idx[a:b]), (X[b:], y[b:], t_idx[b:])

def cap_split(X_, y_, t_, cap=MAX_SAMPLES_SPLIT):
    if len(X_) <= cap: return X_, y_, t_
    idx = np.linspace(0, len(X_) - 1, cap, dtype=int)
    return X_[idx], y_[idx], t_[idx]

def _fmt(ts):
    return ts.strftime("%Y-%m-%d %H:%M") if (ts is not None and pd.notna(ts)) else "—"

def print_split_summary(horizon, t_tr, t_va, t_te):
    def rng(t):
        if len(t)==0: return (None,None,0)
        return (t.min(), t.max(), len(t))
    tr0,tr1,ntr = rng(t_tr); va0,va1,nva = rng(t_va); te0,te1,nte = rng(t_te)
    print(f"[{horizon}m] Split ranges:")
    print(f"  Train: {_fmt(tr0)} → {_fmt(tr1)}  (n={ntr})")
    print(f"  Valid: {_fmt(va0)} → {_fmt(va1)}  (n={nva})")
    print(f"  Test : {_fmt(te0)} → {_fmt(te1)}  (n={nte})")
    return (tr0,tr1),(va0,va1),(te0,te1)

# --------------------------------
# Train PatchTST across horizons
# --------------------------------
results = {}
for horizon in HORIZONS_MIN:
    print(f"\n=== Horizon {horizon} min ===")
    X, y, t_idx = make_windows_from_lagged(df_lag_all, selected_lag_cols,
                                           HISTORY_HOURS, horizon, DT_MINUTES, stride=1)

    if len(t_idx):
        print(f"[{horizon}m] Windows available:", t_idx.min(), "→", t_idx.max(), f"(n={len(t_idx):,})")
    else:
        print(f"[{horizon}m] No windows after lagging/masking."); continue

    mtr = t_idx <= TRAIN_END
    mva = (t_idx > TRAIN_END) & (t_idx <= VAL_END)
    mte = t_idx > VAL_END

    X_tr, y_tr, t_tr = X[mtr], y[mtr], t_idx[mtr]
    X_va, y_va, t_va = X[mva], y[mva], t_idx[mva]
    X_te, y_te, t_te = X[mte], y[mte], t_idx[mte]

    if len(X_tr)==0 or len(X_va)==0 or len(X_te)==0:
        print("  Time split empty; using adaptive 70/15/15.")
        (X_tr, y_tr, t_tr), (X_va, y_va, t_va), (X_te, y_te, t_te) = adaptive_index_split(X, y, t_idx)
    if len(X_te)==0:
        n=len(X); cut=max(1,int(0.85*n))
        X_tr,y_tr,t_tr = X[:cut],y[:cut],t_idx[:cut]
        X_va,y_va,t_va = X[:0],y[:0],t_idx[:0]
        X_te,y_te,t_te = X[cut:],y[cut:],t_idx[cut:]

    _ = print_split_summary(horizon, t_tr, t_va, t_te)

    # scale
    rs     = fit_scaler_from_windows_safe(X_tr, X_va, X_te)
    X_tr_n = norm_windows(X_tr, rs); X_va_n = norm_windows(X_va, rs); X_te_n = norm_windows(X_te, rs)

    # model
    device = torch.device("cuda" if use_gpu else "cpu")
    model = PatchTST(
        in_features=X_tr_n.shape[2], patch_len=PATCH_LEN, stride=PATCH_STRIDE,
        d_model=D_MODEL, nhead=NHEAD, nlayers=NLAYERS, ff_dim=FF_DIM, drop=DROPOUT
    ).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=LR)
    loss_fn = nn.MSELoss(reduction="none")

    ds_tr = TensorDataset(torch.from_numpy(X_tr_n), torch.from_numpy(y_tr))
    ds_va = TensorDataset(torch.from_numpy(X_va_n), torch.from_numpy(y_va))
    dl_tr = DataLoader(ds_tr, batch_size=min(BATCH_SIZE, len(ds_tr)), shuffle=True, drop_last=False)
    dl_va = DataLoader(ds_va, batch_size=min(BATCH_SIZE, len(ds_va)), shuffle=False, drop_last=False)

    best = np.inf; best_state=None; patience=3; bad=0
    q = np.quantile(y_tr, SPIKE_P90) if (WEIGHT_SPIKES and len(y_tr)>0) else None

    for ep in range(EPOCHS):
        model.train(); tr_loss=0.0
        for xb,yb in dl_tr:
            xb,yb = xb.to(device), yb.to(device)
            opt.zero_grad(set_to_none=True)
            with _amp_ctx():
                pred = model(xb)
                l = (pred - yb) ** 2
                if q is not None:
                    w = torch.where(yb >= torch.tensor(q, device=yb.device, dtype=yb.dtype),
                                    torch.tensor(SPIKE_WEIGHT, device=yb.device, dtype=yb.dtype),
                                    torch.tensor(1.0,         device=yb.device, dtype=yb.dtype))
                    l = l * w
                loss = l.mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            opt.step()
            tr_loss += loss.item() * len(xb)
        tr_loss /= max(1, len(dl_tr.dataset))

        model.eval(); va_loss=0.0
        with torch.no_grad(), _amp_ctx():
            for xb,yb in dl_va:
                xb,yb = xb.to(device), yb.to(device)
                out = model(xb)
                va_loss += nn.functional.mse_loss(out, yb, reduction="sum").item()
        va_loss /= max(1, len(dl_va.dataset))
        print(f"[PatchTST] epoch {ep+1:02d}  trainMSE={tr_loss:.5f}  valMSE={va_loss:.5f}")

        if va_loss + 1e-6 < best:
            best = va_loss; best_state = model.state_dict(); bad = 0
        else:
            bad += 1
            if bad >= patience:
                print("[PatchTST] early stop."); break
    if best_state is not None:
        model.load_state_dict(best_state)

    # predict
    def predict_patchtst(model, X):
        ds = TensorDataset(torch.from_numpy(X))
        dl = DataLoader(ds, batch_size=min(BATCH_SIZE, len(ds)), shuffle=False, drop_last=False)
        out=[]; model.eval()
        with torch.no_grad(), _amp_ctx():
            for (xb,) in dl:
                xb = xb.to(device)
                out.append(model(xb).cpu().numpy())
        return np.concatenate(out, axis=0) if len(out)>0 else np.array([])

    yp_va = predict_patchtst(model, X_va_n) if len(X_va_n)>0 else np.array([])
    yp_te = predict_patchtst(model, X_te_n) if len(X_te_n)>0 else np.array([])

    def _rmse(a,b):
        return float(np.sqrt(np.mean((np.asarray(a)-np.asarray(b))**2))) if len(a)==len(b) and len(a)>0 else float("nan")

    R2 = r2_score(y_te, yp_te) if len(yp_te)>0 and len(y_te)>0 else float("nan")
    RMSE = _rmse(y_te, yp_te)
    results[horizon] = dict(t_val=t_va, y_val=y_va, yp_val=yp_va,
                            t=t_te, y=y_te, yp=yp_te, r2=R2, rmse=RMSE)
    print(f"[PatchTST] {horizon}m  Test R²={R2:.3f}  RMSE={RMSE:.3f}")

    # plots
    if PRED_SPLIT.lower() == "val":
        t_plot, y_plot, yhat_plot = t_va, y_va, yp_va; split_name="Validation"
    else:
        t_plot, y_plot, yhat_plot = t_te, y_te, yp_te; split_name="Test"
    if len(t_plot)==0: continue

    mwin = (t_plot >= PRED_START) & (t_plot <= PRED_END)
    if not np.any(mwin):
        end = t_plot[-1]; start = end - pd.Timedelta(days=PLOT_LAST_DAYS)
        mwin = (t_plot >= start) & (t_plot <= end)

    if np.any(mwin):
        ts, yt, ypp = t_plot[mwin], y_plot[mwin], yhat_plot[mwin]
        plt.figure(figsize=(14,4))
        plt.plot(ts, yt, label="Measured", lw=1.2)
        plt.plot(ts, ypp, "--", label=f"Predicted ({split_name})", lw=1.2)
        plt.title(f"PatchTST — {horizon} min ({split_name})")
        plt.ylabel("var1h"); plt.xlabel("Time"); plt.grid(alpha=0.3); plt.legend()
        plt.tight_layout(); plt.show()

print("\nFinal selected lag channels (one per base):")
for nm in selected_lag_cols:
    print("  ", nm)
